
# 📊 ADK Events — Deep Dive

Events are the **atomic units of work** in the Agent Development Kit (ADK). Every input, tool call, model response, state change, artifact save, or control signal is represented as an **Event**. Mastering events gives you full visibility and control over how your agent-based apps behave.

This notebook is a comprehensive, **crisp-yet-detailed** guide to events in ADK with diagrams, tables, and small code snippets you can adapt.



## 📚 Table of Contents

1. [What Events Are and Why They Matter](#section-1)
2. [Understanding and Using Events](#section-2)
   - [Identifying Event Origin and Type](#section-2a)
   - [Extracting Key Information](#section-2b)
   - [Detecting Actions and Side Effects](#section-2c)
   - [Determining if an Event is Final](#section-2d)
3. [How Events Flow: Generation and Processing](#section-3)
4. [Common Event Examples (Illustrative Patterns)](#section-4)
5. [Additional Context and Event Details](#section-5)
6. [Best Practices for Working with Events](#section-6)



<a id="section-1"></a>

## 1) What Events Are and Why They Matter

An **Event** is an immutable record produced during agent execution. It extends a basic LLM response with ADK-specific metadata and **actions** that drive side effects (e.g., state updates, artifact versioning, delegation signals).

**Why events matter**

| Capability | How Events Enable It |
|---|---|
| Communication | Standard format between Runner ⇄ Agents ⇄ Tools ⇄ UI |
| State/Artifact Tracking | `actions.state_delta` and `actions.artifact_delta` capture changes to persist |
| Control Flow | `actions.transfer_to_agent`, `actions.escalate`, `actions.skip_summarization` direct execution |
| Observability | `session.events` is a chronological audit trail for debugging and analytics |

**Conceptual structure (Python-ish)**

```python
# from google.adk.events import Event, EventActions

class Event:
    # --- LLM response-like content ---
    content      # google.genai.types.Content or None
    partial      # bool | None (True during streaming)
    error_code   # Optional[str]
    error_message# Optional[str]

    # --- ADK metadata ---
    author         # "user" or agent/tool name
    invocation_id  # Correlates all events for a single user turn
    id             # Unique event id
    timestamp      # float (epoch seconds)
    branch         # str | None (useful with parallel/workflow agents)

    # --- Side effects & control ---
    actions: EventActions
```

**EventActions (high level)**

- `state_delta: dict` — keys/values to merge into session state
- `artifact_delta: dict[str, int]` — filenames mapped to new version numbers
- `transfer_to_agent: str | None` — delegate to another agent
- `escalate: bool` — stop a loop (e.g., `LoopAgent`)
- `skip_summarization: bool` — deliver raw tool result to the user (bypass LLM summarization)



<a id="section-2"></a>

## 2) Understanding and Using Events

As a developer, you typically **iterate over events** yielded by a `Runner` and infer what's happening to update your UI, logs, or metrics.



<a id="section-2a"></a>

### 2a) Identifying Event Origin and Type

Quick checks that cover 95% of use cases:

| What to check | Meaning |
|---|---|
| `event.author == "user"` | End-user input wrapped by the runner |
| `event.author == "<AgentName>"` | Output from that specific agent/tool step |
| `event.content.parts[0].text` | Text message (streaming if `event.partial` is `True`) |
| `event.get_function_calls()` | LLM requested tool(s) to be executed |
| `event.get_function_responses()` | Tool results fed back to the LLM history |
| `event.actions.state_delta` / `artifact_delta` | Side effects to persist |

```mermaid
flowchart LR
    U[User] -->|User message| R[Runner]
    R -->|Event author=user| H[History]
    A[Agent] -->|LLM/Text| R
    A -->|Tool call request| R
    T[Tool] -->|Tool result| R
    R -->|Append Event| H
    
    
```



<a id="section-2b"></a>

### 2b) Extracting Key Information

**Text content**  
Always guard against `None` and empty parts:

```python
text = None
if event.content and event.content.parts:
    first = event.content.parts[0]
    text = getattr(first, "text", None)
```

**Function calls (tool requests)**

```python
calls = getattr(event, "get_function_calls", lambda: [])()
for call in calls or []:
    tool_name = call.name
    args = call.args  # dict-like
    # Dispatch your tool runner if needed
```

**Function responses (tool results)**

```python
responses = getattr(event, "get_function_responses", lambda: [])()
for r in responses or []:
    tool = r.name
    payload = r.response  # dict returned by the tool
```

**Identifiers & correlation**

- `event.id` — globally unique
- `event.invocation_id` — all events for the same user turn share this
- `event.branch` — hierarchical path (e.g., `Parent.Parallel.Child`) for workflow agents



<a id="section-2c"></a>

### 2c) Detecting Actions and Side Effects

```python
if event.actions:
    if event.actions.state_delta:
        print("State changes ->", event.actions.state_delta)

    if event.actions.artifact_delta:
        print("Artifacts saved ->", event.actions.artifact_delta)

    if getattr(event.actions, "transfer_to_agent", None):
        print("Delegation ->", event.actions.transfer_to_agent)

    if getattr(event.actions, "escalate", False):
        print("Loop termination requested")

    if getattr(event.actions, "skip_summarization", False):
        print("Bypass LLM summarization for tool result")
```



<a id="section-2d"></a>

### 2d) Determining if an Event is a "Final" Response

Use the built-in helper (if available) to avoid re‑creating logic:

```python
if hasattr(event, "is_final_response") and event.is_final_response():
    # Safe to display as final turn output
    ...
```

**Heuristics behind the helper (conceptual):**

```mermaid
flowchart TD
    A[Event arrives] --> B{Has function<br/>responses?}
    B -->|yes & skip_summarization| F[Final raw tool result]
    B -->|yes & no skip| C[Needs LLM summarization]
    B -->|no| D{Has function calls?}
    D -->|yes| C
    D -->|no| E{Is streaming partial?}
    E -->|yes| C
    E -->|no| G[Final text response]
```



<a id="section-3"></a>

## 3) How Events Flow: Generation and Processing

```mermaid
sequenceDiagram
  participant User
  participant Runner
  participant Agent
  participant Tool
  participant SessionService as SessionService (State/Artifacts)

  User->>Runner: Provide input
  Runner->>Runner: Wrap into Event(author='user')
  Runner->>SessionService: append_event(...)
  SessionService-->>Runner: state merged, history updated

  Runner->>Agent: Invoke
  Agent->>Runner: Events (text / function_call / errors)
  Runner->>Tool: Execute when function_call present
  Tool-->>Runner: Event(function_response)

  Runner->>SessionService: append_event(... deltas ...)
  SessionService-->>Runner: Persist changes
  Runner-->>User: Stream events outward
```

Key processing steps:

1. **Generate** an Event (user, agent, tool, or system).
2. **Append** via the configured `SessionService` (merges `state_delta` / `artifact_delta`).
3. **Persist** to `session.events` with IDs/timestamps.
4. **Emit** to your application or UI for rendering.



<a id="section-4"></a>

## 4) Common Event Examples (Illustrative Patterns)

**User Input**

```json
{
  "author": "user",
  "invocation_id": "e-xyz",
  "content": {"parts": [{"text": "Summarize this URL"}]}
}
```

**Agent Streaming Text**

```json
{
  "author": "SummarizerAgent",
  "partial": true,
  "content": {"parts": [{"text": "The article discusses..."}]}
}
```

**Tool Call Request**

```json
{
  "author": "ResearchAgent",
  "content": {"parts": [{"function_call": {"name": "fetch_url", "args": {"url": "https://..."}}}]}
}
```

**Tool Result**

```json
{
  "author": "ResearchAgent",
  "content": {"parts": [{"function_response": {"name": "fetch_url", "response": {"status": 200, "body": "<html>..."}}}]}
}
```

**State Update Only**

```json
{
  "author": "System",
  "content": null,
  "actions": {"state_delta": {"user:plan": "pro"}}
}
```

**Delegation Signal**

```json
{
  "author": "Router",
  "actions": {"transfer_to_agent": "BillingAgent"}
}
```

**Loop Escalation**

```json
{
  "author": "Checker",
  "actions": {"escalate": true}
}
```



<a id="section-5"></a>

## 5) Additional Context and Event Details

**Linking tool requests & results**  
Each LLM function call has an internal **function_call_id**. The `ToolContext` you receive when running a tool includes that ID—useful when multiple tools run in the same turn.

**How state/artifact changes are recorded**  
When you call `context.state[...] = ...` or `await context.save_artifact(...)`, ADK **does not** directly persist. Instead, the framework collects those changes and attaches them to the **next** event’s `actions.state_delta` / `actions.artifact_delta`. The `SessionService.append_event` then applies and persists them.

**State scope reminders**  
Use prefixes appropriately when you put values into state (e.g., `user:`, `app:`, `temp:`) to get the desired persistence/visibility semantics.

**Error events**  
Errors can be represented directly on an Event (e.g., `error_code`, `error_message`) or embedded in a tool response payload. Always guard your UI.



<a id="section-6"></a>

## 6) Best Practices for Working with Events

- **Preserve authorship**: Custom agents should attribute events correctly when yielding.
- **Separate content vs actions**: Use `content` for user-facing data; use `actions` for side effects & control.
- **Lean on helpers**: Prefer `event.is_final_response()` rather than duplicating logic.
- **Exploit history**: `session.events` is your truth for debugging/analytics.
- **Correlate consistently**: Use `invocation_id` to group the turn; `id` for pinpointing a specific event.
- **Be idempotent**: Tooling that mutates external systems should handle duplicates gracefully.
- **UI hygiene**: Stream partial text but only **commit** final text when the final event arrives.


In [ ]:

# PSEUDOCODE — adapt to your app wiring
#
# from google.adk.runners import Runner
# runner = Runner(agent=root_agent, app_name="my_app", session_service=...)
# user_content = Content(parts=[Part(text="Hello!")])
#
# buffer = []
# async for event in runner.run_async(user_id="u1", session_id="s1", new_message=user_content):
#     if event.content and event.content.parts:
#         p0 = event.content.parts[0]
#         if getattr(p0, "text", None):
#             if event.partial:
#                 buffer.append(p0.text)  # stream to UI progressively
#             else:
#                 buffer.append(p0.text)
#     if hasattr(event, "is_final_response") and event.is_final_response():
#         final_text = "".join(buffer).strip()
#         print("FINAL:", final_text or "(non-text final)")
#         buffer.clear()
#     if event.actions and event.actions.state_delta:
#         print("STATE Δ:", event.actions.state_delta)
#     if event.actions and event.actions.artifact_delta:
#         print("ARTIFACT Δ:", event.actions.artifact_delta)
